In [1]:
from tqdm import tqdm
import pandas as pd 
from binance.client import Client
client = Client()

In [2]:
from sqlalchemy import create_engine

In [3]:
coins = ('BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','LUNAUSDT',
  'DOGEUSDT','AVAXUSDT','SHIBUSDT','MATICUSDT','LTCUSDT','UNIUSDT','ALGOUSDT','TRXUSDT',
         'LINKUSDT','MANAUSDT','ATOMUSDT','VETUSDT')

In [4]:
from keys import key

In [15]:
def getminutedata(symbol, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'1m',lookback + ' days ago UTC'))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time','Open','High','Low','Close']
    frame[['Open','High','Low','Close']] = frame[['Open','High','Low','Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time,unit='ms')
    return frame

In [17]:
#init db
engine = create_engine('sqlite:///Cryptoprices.db')

In [18]:
#downloading the coin data 30 days back into respective table inside DB
for coin in tqdm(coins):
    getminutedata(coin,'30').to_sql(coin,engine,index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [15:20<00:00, 46.05s/it]


In [22]:
test = pd.read_sql('BTCUSDT',engine).set_index('Time')

In [54]:
import numpy as np
def technicals(df):
    df = df.copy()
    df = df.resample('60min').ffill()
    df.dropna(inplace=True)
    df['return'] = np.log(df.Close.pct_change()+1)
    df['SMA_fast'] = df.Close.rolling(7).mean()
    df['SMA_slow'] = df.Close.rolling(25).mean()
    df['position'] = np.where(df['SMA_fast'] > df['SMA_slow'],1,0)
    df['strategyreturn'] = df['position'].shift(1)* df['return']  #position * return 
    df.dropna(inplace=True)
    return df

In [55]:
np.exp(technicals(test)[['return','strategyreturn']].sum()) -1

return           -0.190407
strategyreturn   -0.187937
dtype: float64

In [56]:
for coin in coins:
    df = pd.read_sql(coin,engine).set_index('Time')
    print(coin)
    trades = technicals(df).position.diff().value_counts().iloc[1:].sum()
    costs = trades * 0.00075
    print(np.exp(technicals(df)['return'].sum()) -1)
    print(np.exp(technicals(df)['strategyreturn'].sum()) -1- costs)

BTCUSDT
-0.19040678709876113
-0.21793689171205397
ETHUSDT
-0.04369188050226269
-0.08557849059239006
BNBUSDT
-0.020802377414562145
-0.0735539015602851
SOLUSDT
-0.1003738128914935
-0.00022151740508511222
ADAUSDT
-0.31727574750830556
-0.20466841305148872
XRPUSDT
-0.23688711516533623
-0.13760815594244205
DOTUSDT
-0.36705643111336905
-0.30718594166690694
LUNAUSDT
0.6146520146520107
0.005729587204012811
DOGEUSDT
-0.250660792951541
-0.17706288829256875
AVAXUSDT
0.1307169962053112
0.40050544729350496
SHIBUSDT
-0.2867180417044418
-0.025076789238317
MATICUSDT
0.40092470277411074
0.014956843226891912
LTCUSDT
-0.2972714217328869
-0.1602365577818731
UNIUSDT
-0.28128078817734037
-0.19636484246501348
ALGOUSDT
-0.27969328397673177
-0.26073821107691353
TRXUSDT
-0.216284987277353
-0.0941421948939308
LINKUSDT
-0.3220213943194381
-0.2965195967801114
MANAUSDT
-0.1507677989762689
0.12989957179515144
ATOMUSDT
-0.18037735849056558
-0.2671963508077146
VETUSDT
-0.38106870229007606
-0.2801004362097758


In [39]:
technicals(test).position.diff().value_counts().iloc[1:].sum() * 0.00075

1.62975